# Part 1: Create database set
By inputting a target gene, a nested dictionary will be created, with each key being the name of a cancer and each value being a dictionary containing the genomic, proteomic, transcriptomic, and phosphoproteomic data. In other words, the dictionary will be:

{
    cancer : {
                "gen" : <cancer's genomic data>,
                "prot" : <cancer's proteomic data>,
                "trans": <cancer's transcriptomic data>,
                "phos": <cancer's phosphoproteomic data>
            }
    ... for each cancer
}

In [1]:
import cptac
import cptac.pancan as pc
import numpy as np
import cptac.utils as ut
import plot_utils as plt
import pandas as pd

In [2]:
dataset_names = pc.list_datasets(False)
dataset_names  = dataset_names.split('\n')
dataset_names

['PancanBrca',
 'PancanCcrcc',
 'PancanCoad',
 'PancanGbm',
 'PancanHnscc',
 'PancanLscc',
 'PancanLuad',
 'PancanOv',
 'PancanUcec',
 'PancanPdac']

In [3]:
target_gene = "TP53"

In [4]:
datasets = {}
for name in dataset_names:
    exec(f"cancer = pc.{name}(no_internet = True)")  
    genotype_data = cancer.get_genotype_all_vars(mutations_genes = target_gene, omics_source = "washu")
    
    proteomic_data = cancer.get_proteomics(source = 'umich')
    proteomic_data = proteomic_data.droplevel("Database_ID", axis = 1)
    
    phospho_data = cancer.get_phosphoproteomics()
    phospho_data.loc[:, phospho_data.columns.get_level_values(0) == "TP53"].droplevel(["Database_ID", "Peptide"], axis = 1)
    
    transcript_data = cancer.get_transcriptomics(source = "washu")
    transcript_data = transcript_data.droplevel("Database_ID", axis = 1)
    
    datasets[name[6:]] = {
                        "gen" : genotype_data,
                        "prot": proteomic_data,
                        "trans": transcript_data,
                        "phos" : phospho_data
                         }    

/home/sdsquires/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
/home/sdsquires/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 68 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)
cptac warning: washu_CNV data was not found for the following samples, so washu_CNV data columns were filled with NaN for these samples: 01BR044, 604 (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 104 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 44 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 67 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


...

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 9 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 50 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 5 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 73 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 41 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


# Part 2: Select mutations
Determine which cancers have mutation types with rates above 20%.

To facilitate that, the mutation types from the get_genotype_all_vars must be grouped into functional types.


In [ ]:
mutation_types = ["Deletion", "Indel", "Missense", "Wildtype"]

In [5]:
# Grouping the mutation types for each cancer
for name, dataset in datasets.items():
    genotypes = dataset["gen"]
    genotypes.replace(["Nonsense_Mutation", "Frame_Shift_Ins", "Frame_Shift_Del"], "Truncation", inplace = True)
    genotypes.replace(["In_Frame_Del", "In_Frame_Ins"], "Indel", inplace = True)
    genotypes.replace(["Missense_Mutation", "Wildtype_Tumor"], ["Missense", "Wildtype"], inplace = True)
    genotypes = genotypes.loc[genotypes["Mutation"] != "Silent"]
    datasets[name]["gen"] = genotypes


In [ ]:
# Create a bargraph displaying the mutation types

In [ ]:
# Determine which cancers have mutation rates over 20%
variants_to_analyze = {}
for name, dataset in datasets.items():
    mut_type_freq = dataset["gen"]["Mutation"].value_counts() / len(dataset["prot"])
    mut_type_freq = mut_type_freq.loc[mut_type_freq >= 0.2]
    variants_to_analyze[name] = set(mut_type_freq.index.tolist())
    variants_to_analyze[name].discard("Wildtype")
print(str(variants_to_analyze))

# Part 3: Cis effects
Determine which cancers alter their protein expression level of the target gene due to a mutation in that gene (with Wildtype as a control).
Writes out a file containing the infomation for each mutation and cancer, and creates boxplot summarizing the data

In [ ]:
# Create a function that will analyze the protein cis effects for a given mutation type
def analyze_mutation(mutation_type, write_out = True, pval_only = True):
    """Analyzes the cis effects for a given mutation type across all cancers.
    
    Parameters:
    mutation_type (list[str]): The mutation type to analyze, as a string. Possible values given in mutation_types.
    write_out (bool, optional): Whether to write out the data to a separate file.
    pval_only (bool, optional): Whether to only return the p_value from the analysis, or both the t-statistics and p_val.
    """
    output_string = ""
    for name, dataset in datasets.items():
        if mutation_type not in variants_to_analyze[name]: continue
        genotype_and_prot = pd.merge(dataset["gen"], dataset["prot"], left_index = True, right_index = True)
        mutation = genotype_and_prot.loc[genotype_and_prot["Mutation"] == mutation_type][target_gene].dropna()
        wildtype = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Wildtype"][target_gene].dropna()
        result = ttest_ind(mutation, wildtype)
        
        if not pval_only:
            output_string += f"{name}\nNumber of {mutation_type}s = {len(mutation)}"
            output_string += f"Number of Wildtypes = {len(wildtype)}\n"
        output_string += f"{name} {mutation_type} P-value = {result[1]}\n\n"
        
    return output_string    


In [6]:
target_dataset = datasets["Ccrcc"]
target_dataset[2]
# target_dataset[2].loc[target_dataset["Mutation"] == "Deletion"]

Name,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,5S_rRNA,...,uc_338,uc_338,uc_338,uc_338,uc_338,uc_338,uc_338,uc_338,uc_338,yR211F11.2
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00004,0.0,0.0,0.0,3.354233,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,0.718764,1.796911,0.811508,5.616391,0.000000,0.0,0.370406,0.0,0.000000,0.104097
C3L-00010,0.0,0.0,0.0,2.350325,0.000000,0.192569,0.0,0.0,0.000000,0.0,...,0.545611,0.136403,0.739215,5.462455,0.000000,0.0,1.687043,0.0,0.000000,0.316078
C3L-00011,0.0,0.0,0.0,5.843757,0.412252,0.000000,0.0,0.0,0.000000,0.0,...,0.262565,0.000000,0.000000,6.155015,0.129738,0.0,0.135310,0.0,0.000000,0.190133
C3L-00026,0.0,0.0,0.0,1.445766,0.451681,0.203067,0.0,0.0,0.000000,0.0,...,0.863034,0.000000,0.389757,5.619753,0.000000,0.0,0.444753,0.0,0.129919,0.499964
C3L-00079,0.0,0.0,0.0,1.111766,2.674472,0.000000,0.0,0.0,0.000000,0.0,...,2.167943,0.000000,0.419602,1.815022,0.000000,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01646.N,0.0,0.0,0.0,2.043907,0.957825,0.287079,0.0,0.0,0.000000,0.0,...,0.813391,0.203348,2.571366,4.568233,0.000000,0.0,2.724612,0.0,0.000000,0.294504
C3N-01648.N,0.0,0.0,0.0,0.570147,1.870295,0.000000,0.0,0.0,0.000000,0.0,...,2.580933,0.000000,0.896602,6.011403,0.000000,0.0,3.887842,0.0,1.613883,0.632568
C3N-01649.N,0.0,0.0,0.0,0.704438,1.797306,0.000000,0.0,0.0,0.256758,0.0,...,0.981182,0.000000,1.329344,7.347457,0.000000,0.0,2.865293,0.0,0.443115,0.426307


In [12]:
from scipy.stats import ttest_ind
with open("deletion_protein_data.txt", 'w') as outFile:
    for name, dataset in datasets.items():
        if name not in deletion_sets: continue
        outFile.write(name + '\n')
        genotype_and_prot = dataset[0].merge(dataset[1], left_index = True, right_index = True)
        deletions = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Deletion"][target_gene].dropna()
        wildtypes = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Wildtype_Tumor"][target_gene].dropna()
        outFile.write("len(deletions) = " + str(len(deletions)) + '\n')
        outFile.write("len(wildtypes) = " + str(len(wildtypes)) + '\n')
        result = ttest_ind(deletions, wildtypes)
        outFile.write(name + " deletion P-value: " + str(result[1]) + '\n')
        outFile.write("\n")
        

In [13]:
#Cancers with more than 20% having Missense and Wildtype
missense_sets = ["Luad", "Pdac"]
for name, dataset in datasets.items():
    if name not in missense_sets: continue
    print(name)
    genotype_and_prot = dataset[0].merge(dataset[1], left_index = True, right_index = True)
    missenses = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Missense_Mutation"][target_gene].dropna()
    wildtypes = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Wildtype_Tumor"][target_gene].dropna()
    print("len(missenses) = " + str(len(missenses)))
    print("len(wildtypes) = " + str(len(wildtypes)))
    result = ttest_ind(deletions, wildtypes)
    print(name + " missense P-value: " + str(result[1]))
    print("\n")
    
# genotype_and_prot = datasets["Luad"][1].merge(datasets["Luad"][2], left_index = True, right_index = True)
# genotype_and_prot["Mutation"]
# genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Missense_Mutation"]

Luad
len(missenses) = 21
len(wildtypes) = 29
Luad missense P-value: 0.000165611279181047


Pdac
len(missenses) = 28
len(wildtypes) = 21
Pdac missense P-value: 0.008603505603947332




Trans effects

In [10]:
paths = ut.get_pathways_with_proteins(proteins = target_gene, database = 'reactome')
paths

,id,pathway,pathway_id
0,TP53,Activation of NOXA and translocation to mitoch...,R-HSA-111448
1,TP53,Activation of PUMA and translocation to mitoch...,R-HSA-139915
2,TP53,Pre-NOTCH Transcription and Translation,R-HSA-1912408
3,TP53,Oxidative Stress Induced Senescence,R-HSA-2559580
4,TP53,Formation of Senescence-Associated Heterochrom...,R-HSA-2559584
5,TP53,Oncogene Induced Senescence,R-HSA-2559585
6,TP53,DNA Damage/Telomere Stress Induced Senescence,R-HSA-2559586
7,TP53,SUMOylation of transcription factors,R-HSA-3232118
8,TP53,Autodegradation of the E3 ubiquitin ligase COP1,R-HSA-349425
9,TP53,Association of TriC/CCT with target proteins d...,R-HSA-390471


In [45]:
interacting_proteins = ut.get_proteins_in_pathways(paths["pathway_id"], database = 'reactome')
interacting_proteins = list(interacting_proteins["member"])

genes_measured = list(datasets["Brca"][1].columns)
new_genes = sorted(list(set([protein for protein in interacting_proteins if protein in genes_measured])))

In [47]:
interacting_proteins = set(interacting_proteins)
print(len(interacting_proteins))
genes_measured = set(genes_measured)
print(len(genes_measured))
print(len(new_genes))
new_genes

1077
12046
885


['ABL1',
 'ABRAXAS1',
 'ACD',
 'ACTB',
 'ADRM1',
 'AGO1',
 'AGO2',
 'AGO3',
 'AGO4',
 'AIFM2',
 'AK3',
 'AKAP1',
 'AKAP10',
 'AKT1',
 'AKT2',
 'AKT3',
 'ALOX15',
 'ALOX5',
 'ANAPC1',
 'ANAPC10',
 'ANAPC11',
 'ANAPC15',
 'ANAPC16',
 'ANAPC2',
 'ANAPC4',
 'ANAPC5',
 'ANAPC7',
 'ANXA1',
 'AP3M1',
 'APAF1',
 'APBB1',
 'APC',
 'AR',
 'ARFGEF2',
 'ARID3A',
 'ARRB1',
 'ARRB2',
 'ASF1A',
 'ATF2',
 'ATM',
 'ATN1',
 'ATR',
 'ATRIP',
 'ATXN7',
 'AURKA',
 'AURKB',
 'AXIN1',
 'BABAM1',
 'BABAM2',
 'BAK1',
 'BANP',
 'BAP1',
 'BARD1',
 'BATF',
 'BAX',
 'BAZ1B',
 'BBC3',
 'BCL2',
 'BCL2L1',
 'BCL2L14',
 'BCL6',
 'BECN1',
 'BIRC2',
 'BIRC3',
 'BIRC5',
 'BLM',
 'BNIP3L',
 'BRCA1',
 'BRCC3',
 'BRD1',
 'BRD7',
 'BRIP1',
 'BRPF1',
 'BRPF3',
 'BTG2',
 'CABIN1',
 'CABLES1',
 'CABLES2',
 'CAPZA1',
 'CAPZA2',
 'CAPZB',
 'CARM1',
 'CARMIL1',
 'CASP1',
 'CASP10',
 'CASP2',
 'CASP3',
 'CASP4',
 'CASP5',
 'CASP6',
 'CBX2',
 'CBX4',
 'CBX5',
 'CBX6',
 'CBX8',
 'CCNA2',
 'CCNB1',
 'CCNB2',
 'CCND1',
 'CCNE1',
 'CCNE

In [32]:
for dataset in datasets.values():
    print(dataset[1].iloc[:,0:5])

Name            A1BG       A2M     A2ML1  A4GALT      AAAS
Patient_ID                                                
01BR001     0.964219  0.712229 -1.838762     NaN  0.190146
01BR008    -0.163783  0.222128  0.564935     NaN  0.219314
01BR009     0.440711  1.305547  1.064684     NaN -0.051330
01BR010    -0.210077 -0.476778 -2.153951     NaN  0.151513
01BR015     0.512848 -0.108772 -2.024479     NaN  0.428309
...              ...       ...       ...     ...       ...
21BR010     0.525813  0.171819 -2.162125     NaN -0.034245
22BR005     0.072730 -0.265281 -2.320531     NaN  0.070773
22BR006     0.171588 -0.177432 -2.140071     NaN  0.247685
CPT000814  -0.092933  0.105472  1.614023     NaN  0.891093
CPT001846   0.877408  0.840633 -1.705519     NaN  0.582961

[125 rows x 5 columns]
Name             A1BG      A1CF       A2M  A4GALT      AAAS
Patient_ID                                                 
C3L-00004   -0.607491  0.569688 -0.104659     NaN  0.213441
C3L-00010    0.909146  0.2898

In [36]:
with open("deletion_trans_data.txt", 'w') as outFile:
    all_significant = {}
    for name, dataset in datasets.items():
        if name not in deletion_sets: continue
        print(name + '\n')
        outFile.write(name + '\n')
        significant_changes = {}
        for trans_gene in interacting_proteins:
            genotype_and_prot = dataset[0].merge(dataset[1], left_index = True, right_index = True)
            if trans_gene not in genotype_and_prot: continue
            deletions = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Deletion"][trans_gene].dropna()
            wildtypes = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Wildtype_Tumor"][trans_gene].dropna()
            result = ttest_ind(deletions, wildtypes)
            print(trans_gene + " P-value with " + name + " deletion: " + str(result[1]))
            outFile.write(name + " deletion P-value: " + str(result[1]))
        print('\n')
        outFile.write('\n')


Brca

CDK9 P-value with Brca deletion: 0.50616170130435
ADRM1 P-value with Brca deletion: 0.2570161604492979
IDE P-value with Brca deletion: 0.01963302926516782
PPP1R13B P-value with Brca deletion: 0.37996315163549066
PRKAR1B P-value with Brca deletion: 0.2079770723392689
YWHAE P-value with Brca deletion: 1.8048453078396805e-06
RCE1 P-value with Brca deletion: 0.046742124329750855
HMGA2 P-value with Brca deletion: 0.6556806610560537
STAT1 P-value with Brca deletion: [0.48100708 0.99206345]
PSMB3 P-value with Brca deletion: 0.47275441207401236
RB1 P-value with Brca deletion: 0.253574758582502
GAPDHS P-value with Brca deletion: 0.2539753536455864
RRAGC P-value with Brca deletion: 0.004848254919865369
TP53BP1 P-value with Brca deletion: 0.004904163121077241
NDRG1 P-value with Brca deletion: 0.025479713514701803
ANXA1 P-value with Brca deletion: 0.13828433775035992
HIRA P-value with Brca deletion: 0.6121421608801694
JUN P-value with Brca deletion: 0.9575012115374687
KIFC3 P-value with Brca

SESN2 P-value with Brca deletion: 0.06418123030221848
RBL2 P-value with Brca deletion: 0.07305648329805116
TAF11 P-value with Brca deletion: 0.7009423936747834
CHD4 P-value with Brca deletion: [0.14889302 0.32947588]
LAMTOR2 P-value with Brca deletion: 0.956068395327735
POLR2L P-value with Brca deletion: 0.06889726626167673
GTF2H5 P-value with Brca deletion: 0.7984259700145018
USP24 P-value with Brca deletion: 0.09070497186069192
HIPK1 P-value with Brca deletion: 0.687642039946117
YWHAQ P-value with Brca deletion: 0.04563487708114604
ANAPC1 P-value with Brca deletion: 0.5010260276117011
ANAPC7 P-value with Brca deletion: 0.6944259233712768
YWHAZ P-value with Brca deletion: 0.009133109747234434
NOS2 P-value with Brca deletion: nan
PHC2 P-value with Brca deletion: 0.5530323550467631
NUAK1 P-value with Brca deletion: 0.43123162953956906
BIRC5 P-value with Brca deletion: 0.06429876061981726
RBBP7 P-value with Brca deletion: [0.89625937 0.02596675]
PSMB2 P-value with Brca deletion: 0.224047

ING2 P-value with Brca deletion: 0.7675210860836644
BIRC2 P-value with Brca deletion: 0.6243088368911969
CHM P-value with Brca deletion: 0.8889200213731796
KIF13A P-value with Brca deletion: [0.13626457 0.06531282]
E2F4 P-value with Brca deletion: 0.09498353797839314
CSNK2A1 P-value with Brca deletion: 0.07847610025781532
RMI2 P-value with Brca deletion: 0.6936140981027386
DCAF7 P-value with Brca deletion: 0.2702299149267436
PRKAR1A P-value with Brca deletion: 0.06434056805307313
YWHAG P-value with Brca deletion: 0.9681292533607754
MAPKAPK3 P-value with Brca deletion: 0.5614676799004257
ARRB1 P-value with Brca deletion: [0.0202926  0.80277234]
CDK13 P-value with Brca deletion: [0.6600761  0.18304468]
BNIP3L P-value with Brca deletion: 0.39284020223669525
EZH2 P-value with Brca deletion: 0.026998857843554394
SMAD3 P-value with Brca deletion: 0.8849620436089785
TP53BP2 P-value with Brca deletion: 0.8865602095338678
USP10 P-value with Brca deletion: 0.7632302919960332
PRKAR2B P-value with

EGR1 P-value with Brca deletion: nan
PSME2 P-value with Brca deletion: 0.005215833615292014
CCNA2 P-value with Brca deletion: 0.00036551926223601705
LEF1 P-value with Brca deletion: 0.5785003384051306
KAT6A P-value with Brca deletion: 0.5500153113807125
MAP4K4 P-value with Brca deletion: [0.0590573  0.15334504 0.0149478  0.25418122 0.31451118]
DOCK2 P-value with Brca deletion: 0.0037857981384012817
CASP10 P-value with Brca deletion: 0.00839522884688249
CCT7 P-value with Brca deletion: 0.46870258794075736
PRKAB1 P-value with Brca deletion: 0.043329658008575564
UBE2N P-value with Brca deletion: 0.6457022995715627
TAF7 P-value with Brca deletion: 0.9587408126072473
TNKS P-value with Brca deletion: 0.6992083568863563
RHOA P-value with Brca deletion: 0.07199193951333237
MBD3 P-value with Brca deletion: 0.9757737099378334
ZEB1 P-value with Brca deletion: 0.7020445812814627
MNAT1 P-value with Brca deletion: 0.46015739495825125
PSMD3 P-value with Brca deletion: 0.8299986011469446
USP16 P-value

MAPKAP1 P-value with Brca deletion: 0.06926515430063954
HMGB1 P-value with Brca deletion: 0.2605902410536426
H1-4 P-value with Brca deletion: 0.2861693116767856
PIN1 P-value with Brca deletion: 0.7844512232648312
H2BC18 P-value with Brca deletion: 0.0339394814112802
CBX6 P-value with Brca deletion: 0.020202990177218577
USP21 P-value with Brca deletion: 0.8067594288941151
AGO2 P-value with Brca deletion: 0.08240538684739492
NFKB1 P-value with Brca deletion: 0.92780433428873
ANAPC2 P-value with Brca deletion: 0.07305595276468774
PSMB4 P-value with Brca deletion: 0.33296423665720376
LAMTOR4 P-value with Brca deletion: 0.360985072514182
SKP2 P-value with Brca deletion: 0.38192299367399674
SMAD1 P-value with Brca deletion: 0.1300991205960766
TADA2B P-value with Brca deletion: 0.5827146780024393
SESN3 P-value with Brca deletion: 0.8134231574983969
ETS1 P-value with Brca deletion: 0.018085967414505924
TINF2 P-value with Brca deletion: 0.3633156291535603
ITPK1 P-value with Brca deletion: 0.509

...

AKAP10 P-value with Pdac deletion: 0.11454399028837944
MAPK3 P-value with Pdac deletion: 0.41481420033401684
WDR20 P-value with Pdac deletion: 0.03774130567952178
DYRK2 P-value with Pdac deletion: 0.7973314902333154
TIGAR P-value with Pdac deletion: 0.624817094883904
LAMTOR3 P-value with Pdac deletion: 0.4993116271860575
IL1B P-value with Pdac deletion: 0.6514441189312999
BRCC3 P-value with Pdac deletion: 0.7036934598379059
PSMA5 P-value with Pdac deletion: 0.014234306732576089
ANAPC16 P-value with Pdac deletion: 0.08109250882094943
CDKN2D P-value with Pdac deletion: 0.09228254606386738
USP22 P-value with Pdac deletion: 0.9291609324488521
RFC4 P-value with Pdac deletion: 0.06762853431860309
KIF5A P-value with Pdac deletion: 0.666473999086251
SNAI1 P-value with Pdac deletion: 0.08371647834128748
NDUFA4 P-value with Pdac deletion: 0.2760536756988151
MAFF P-value with Pdac deletion: 0.08826823830266677
FKBPL P-value with Pdac deletion: 0.1334121731669296
EHD2 P-value with Pdac deletion: 0

BCL2L1 P-value with Pdac deletion: 0.0003343611441346198
GTF2F2 P-value with Pdac deletion: 0.4059202059562539
ORC2 P-value with Pdac deletion: 0.8791232749060922
USP5 P-value with Pdac deletion: 0.9000585293482248
TRAF6 P-value with Pdac deletion: 0.2948267844204024
KIF13B P-value with Pdac deletion: 0.003240229589511109
BRPF3 P-value with Pdac deletion: 0.14159872321654404
JUNB P-value with Pdac deletion: 0.2112249335443583
STK11 P-value with Pdac deletion: 0.2283810728999828
KLC1 P-value with Pdac deletion: [nan nan nan]
KDM4B P-value with Pdac deletion: 0.7314683584529904
CCNT2 P-value with Pdac deletion: 0.1103248548390669
MAPK1 P-value with Pdac deletion: 0.0026459926003900734
CNOT4 P-value with Pdac deletion: [nan nan]
DOCK11 P-value with Pdac deletion: 4.806409035912932e-06
MAP2K6 P-value with Pdac deletion: 0.18349780150230935
TUBB2B P-value with Pdac deletion: 0.6895784865207406
ELF3 P-value with Pdac deletion: 0.005946285544209905
PPP5C P-value with Pdac deletion: 8.49205025

G6PD P-value with Pdac deletion: 0.009884952240075367
LAMTOR5 P-value with Pdac deletion: 0.3331759354887669
ANAPC11 P-value with Pdac deletion: 0.36582120352639846
FKBP8 P-value with Pdac deletion: [0.00103845 0.06857754]
CDKN2A P-value with Pdac deletion: [nan nan]
AGO4 P-value with Pdac deletion: 0.992833436275073
SNX3 P-value with Pdac deletion: 4.968044075611227e-05
ITGB1 P-value with Pdac deletion: 0.02534369378215619
CBX5 P-value with Pdac deletion: 0.14254055327762621
DDB2 P-value with Pdac deletion: 0.01948117222014082
HIC1 P-value with Pdac deletion: 0.0006687825011301638
H1-2 P-value with Pdac deletion: 0.014218853472922233
GZMB P-value with Pdac deletion: 0.8150317030607618
KIF15 P-value with Pdac deletion: 0.22985403507166086
SCMH1 P-value with Pdac deletion: 0.1015167459083035
BIRC3 P-value with Pdac deletion: 0.26371615418728866
MECOM P-value with Pdac deletion: 0.004192185180532023
PSMC6 P-value with Pdac deletion: 0.01896965854065422
SESN1 P-value with Pdac deletion: 0

TP53RK P-value with Pdac deletion: 0.0010904818940501877
HERC2 P-value with Pdac deletion: 0.9986690680661736
SP3 P-value with Pdac deletion: 0.7479097851499361
TERF1 P-value with Pdac deletion: 0.07338895503611094
KIF16B P-value with Pdac deletion: 0.40875593525927467
PIP4P1 P-value with Pdac deletion: 0.46777057840283787
TRAF3 P-value with Pdac deletion: 0.04930812461522029
BRD1 P-value with Pdac deletion: 0.02518436451769584
CARM1 P-value with Pdac deletion: 0.343995844349247
RNF146 P-value with Pdac deletion: 0.16283179686128582
NELFB P-value with Pdac deletion: 0.16289763730665355
KIF2A P-value with Pdac deletion: [0.64342884 0.30448027]
IGHE P-value with Pdac deletion: 0.08878887220526953
TNKS1BP1 P-value with Pdac deletion: 0.00936737791101073
OTUB1 P-value with Pdac deletion: 0.24207197961080087
POLR2G P-value with Pdac deletion: 0.016549729085980954
VDAC2 P-value with Pdac deletion: 0.028958371384576603
MAPKAP1 P-value with Pdac deletion: 0.13512904016107885
HMGB1 P-value with

CNOT10 P-value with Pdac deletion: 0.8638789661678246
MYC P-value with Pdac deletion: 0.5077767049238472
BECN1 P-value with Pdac deletion: 0.026152702341453022
CTDP1 P-value with Pdac deletion: 0.06549252324748038
UBN1 P-value with Pdac deletion: 0.01191719568375687
LAMTOR1 P-value with Pdac deletion: 0.003895344080236377
ELOB P-value with Pdac deletion: 0.004569079356124619
MAP2K4 P-value with Pdac deletion: 0.0004421973050857271
UBXN1 P-value with Pdac deletion: 0.030077285541564698
PRKAA2 P-value with Pdac deletion: 0.5251497064155054
STAT6 P-value with Pdac deletion: 0.4450509243677848
HSP90B1 P-value with Pdac deletion: 0.009262287241724036
HUS1 P-value with Pdac deletion: 0.9107773232290493
NOP56 P-value with Pdac deletion: 0.004935402662545024
USP30 P-value with Pdac deletion: 0.996593114680032
RUNX1 P-value with Pdac deletion: 0.6395563817299968
AGO1 P-value with Pdac deletion: 0.017095604386683165
MAPK10 P-value with Pdac deletion: 0.21865921152988796
ID1 P-value with Pdac del